In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import torch as th
from syft import VirtualMachine
from pathlib import Path
from torchvision import datasets, transforms
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, build_plan_inputs, ROOT_CLIENT
from syft.lib.python.list import List
from matplotlib import pyplot as plt
from syft import logger
from syft.lib.torch.module import ModelExecutor
logger.remove()

In [3]:
alice = VirtualMachine()
alice_client = alice.get_root_client()

# Define Model

In [4]:
class MLP(th.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = th.nn.Linear(784, 100)
        self.a1 = th.nn.ReLU()
        self.l2 = th.nn.Linear(100, 10)
        
    @staticmethod
    def forward(model, x):
        l1_out = model.a1(model.l1(x))
        l2_out = model.l2(l1_out)
        return l2_out

In [5]:
local_model = MLP()

In [6]:
executor = ModelExecutor(local_model)

# Define Plan

In [7]:
dummy_dl = sy.lib.python.List([
    [th.rand([64,784]), th.randint(0,10, (64,))] for _ in range(1)
])

In [8]:
@make_plan
def train(dl=dummy_dl,
          model=local_model
         ):
    remote_torch = ROOT_CLIENT.torch
    optimizer = remote_torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0)
    criterion = remote_torch.nn.CrossEntropyLoss()
    
    for xy in dl:
        x = xy[0]
        y = xy[1]
        out = executor(model,x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        
    return loss, model

/mnt/d/ubuntu/anaconda3/envs/pysyft_dev/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


# Run

In [9]:
remote_model = local_model.send(alice_client)
dummy_dl_ptr = dummy_dl.send(alice_client)
train_ptr = train.send(alice_client)

In [10]:
res_ptr = train_ptr(dl=dummy_dl_ptr, model=remote_model)

KeyError: <UID: 172629a8d97d49ada80e6fbc88df7f6e>